
# Black-Scholes Option Pricing

## Intuition
The Black-Scholes model prices European options by **replicating** the option payoff
with a dynamic portfolio of the underlying stock and a risk-free bond. If a
portfolio perfectly replicates the option, arbitrage implies they must have the
same price.

A key result is that the option can be priced under the **risk-neutral measure**,
which replaces the real drift $\mu$ with the risk-free rate $r$.

## Model Assumptions
- Frictionless markets, no arbitrage
- Constant volatility $\sigma$ and interest rate $r$
- Lognormal stock dynamics
- European exercise (only at maturity)

## Stock Dynamics

```{math}
\mathrm{d}S_t = \mu S_t\,\mathrm{d}t + \sigma S_t\,\mathrm{d}W_t
```

## Closed-Form Prices
Call:

```{math}
C = S_0 N(d_1) - K e^{-rT} N(d_2)
```

Put:

```{math}
P = K e^{-rT} N(-d_2) - S_0 N(-d_1)
```

where

```{math}
 d_1 = \frac{\ln(S_0 / K) + (r + \tfrac{1}{2}\sigma^2)T}{\sigma\sqrt{T}},
 \quad
 d_2 = d_1 - \sigma\sqrt{T}
```

Put-call parity:

```{math}
C - P = S_0 - K e^{-rT}
```

In [ ]:
import numpy as np
import plotly.graph_objects as go
from IPython.display import HTML

def show_plot(fig):
    return HTML(fig.to_html(include_plotlyjs=False, full_html=False))


In [ ]:
def norm_cdf(x):
    return 0.5 * (1.0 + np.erf(x / np.sqrt(2.0)))

def black_scholes_call(S, K, r, sigma, T):
    S = np.asarray(S, dtype=float)
    sigma = np.asarray(sigma, dtype=float)
    T = np.asarray(T, dtype=float)
    eps = 1e-12
    sigma = np.maximum(sigma, eps)
    T = np.maximum(T, eps)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm_cdf(d1) - K * np.exp(-r * T) * norm_cdf(d2)

def black_scholes_put(S, K, r, sigma, T):
    S = np.asarray(S, dtype=float)
    sigma = np.asarray(sigma, dtype=float)
    T = np.asarray(T, dtype=float)
    eps = 1e-12
    sigma = np.maximum(sigma, eps)
    T = np.maximum(T, eps)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm_cdf(-d2) - S * norm_cdf(-d1)

def black_scholes_delta(S, K, r, sigma, T, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        return norm_cdf(d1)
    return norm_cdf(d1) - 1.0

def black_scholes_vega(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    pdf = (1.0 / np.sqrt(2.0 * np.pi)) * np.exp(-0.5 * d1**2)
    return S * pdf * np.sqrt(T)



## Payoff at Expiration
Payoff is the option value at $T$. This is **not** the same as today's price.

In [ ]:
S_grid = np.linspace(50, 150, 200)
K = 100
call_payoff = np.maximum(S_grid - K, 0)
put_payoff = np.maximum(K - S_grid, 0)

fig = go.Figure()
fig.add_trace(go.Scatter(x=S_grid, y=call_payoff, mode='lines', name='Call Payoff'))
fig.add_trace(go.Scatter(x=S_grid, y=put_payoff, mode='lines', name='Put Payoff'))
fig.update_layout(
    title='Payoff at Expiration',
    xaxis_title='Underlying Price (S)',
    yaxis_title='Payoff',
    template='plotly_white'
)
show_plot(fig)



## Price vs Underlying
As $S$ increases, call value rises and put value falls.

In [ ]:
r = 0.03
sigma = 0.2
T = 1.0

call_prices = black_scholes_call(S_grid, K, r, sigma, T)
put_prices = black_scholes_put(S_grid, K, r, sigma, T)

fig = go.Figure()
fig.add_trace(go.Scatter(x=S_grid, y=call_prices, mode='lines', name='Call Price'))
fig.add_trace(go.Scatter(x=S_grid, y=put_prices, mode='lines', name='Put Price'))
fig.update_layout(
    title='Black-Scholes Price vs Underlying',
    xaxis_title='Underlying Price (S)',
    yaxis_title='Option Price',
    template='plotly_white'
)
show_plot(fig)



## Price Surface (S and Volatility)
Higher volatility increases both call and put prices.

In [ ]:
S_vals = np.linspace(50, 150, 60)
sig_vals = np.linspace(0.05, 0.8, 60)
S_mesh, sig_mesh = np.meshgrid(S_vals, sig_vals)
C_surface = black_scholes_call(S_mesh, K, r, sig_mesh, T)

fig = go.Figure(data=[go.Surface(x=S_mesh, y=sig_mesh, z=C_surface)])
fig.update_layout(
    title='Call Price Surface',
    scene=dict(xaxis_title='S', yaxis_title='Volatility', zaxis_title='Call Price'),
    template='plotly_white'
)
show_plot(fig)



## Greeks (Delta and Vega)
- **Delta** measures sensitivity to the underlying price.
- **Vega** measures sensitivity to volatility.

In [ ]:
delta_call = black_scholes_delta(S_grid, K, r, sigma, T, option_type='call')
vega = black_scholes_vega(S_grid, K, r, sigma, T)

fig = go.Figure()
fig.add_trace(go.Scatter(x=S_grid, y=delta_call, mode='lines', name='Call Delta'))
fig.add_trace(go.Scatter(x=S_grid, y=vega, mode='lines', name='Vega'))
fig.update_layout(
    title='Greeks vs Underlying',
    xaxis_title='Underlying Price (S)',
    yaxis_title='Sensitivity',
    template='plotly_white'
)
show_plot(fig)
